# Getting started with PySkinDose
Welcome! This is a getting-started guide for PySkinDose. It will be updated when more functionality is added or modified.



## PART I: Settings

We need to start our session with adding the class PyskindoseSettings for parsing of user defined settings:

In [ ]:
import json
from pyskindose.settings_pyskindose import PyskindoseSettings
from pyskindose.main import main

#import plotly.io as pio
#pio.renderers.default = "browser"

once completed, we need to initialize all the user defined settings. Lets load a template of pre-loaded settings  (located in settings_example.json), then we can change each of the inidividual settings prior to calculating skin dose

In [ ]:
# load the settings.json file
settings_json = json.load(open("../settings_example.json"))

# Parse the settings to a setting class:
settings = PyskindoseSettings(settings=settings_json)

Now, all the settings are populated in settings. We can print all user defined setting by typing `settings.print_parameters()`:

In [ ]:
settings.print_parameters()

As seen in the above output, `settings`  includes the sections __general__, __phantom__, and __plot__. 

You can access each of the settings in general by simply typing `settings.mode`, `settings.k_tab_val` etc. For the other sections, include the section name, e.g. `settings.phantom.patient_orientation`.


You can read up each by adressing the corresponding docstring with the `__doc__` function. E.g: `settings__doc__` returns detailed descriptions of the setting in __general__, i.e., "mode", "k_tab_val",  "estimate_k_tab", and "rdsr_filename". 

Uncommany any of the following lines for more info on each of the subsections in settings.


In [ ]:
# UNCOMMENT ANY OF THE FOLLOWING LINES FOR DESCRIPTION ON THE VARIABLES IN SETTINGS

#print(settings.__doc__) # uncomment this line to read up on settings.general
#print(settings.phantom.__doc__) # uncomment this line to read up on settings.phantom
#print(settings.phantom.patient_offset.__doc__)  # uncomment this line to read up on settings.phantom.patient_offset
#print(settings.phantom.dimension.__doc__) # uncomment this line to read up on settings.phantom.dimension
#print(settings.plot.__doc__) # uncomment this line to read up on settings.plot


## PART II: Plot setup

In [ ]:
#This section uses a lot of quite memory heavy interactive .html plots. 
#If you are low on available RAM memory, uncommen the following code block. 
#This will export all the plots to separate browser tabs so that you can close the each plot prior to creating a new one

#import plotly.io as pio
#pio.renderers.default = "browser"

Now, lets have a look on the geometry in which the skin dose is calculated. PySkinDose has a built in mode for plotting the setup (i.e. patient phantom, and its position upon the patient support table. This is accessed by running `main.py` with `settings.mode` = "plot_setup". Since mode is already set to "plot_setup" i setting, we simply run main like this:

In [ ]:
main(settings=settings)

In the above output, you will see the patient phantom and its position upon the support table. The X-ray source, beam, and image receptor is also added in standard position (AP1=AP2=0)
There are a few things one should adress prior to conducting any skin dose estimation.

1. **Patient phantom selection**: Select you phantom by setting `settings.phantom.phantom_type` to either "plane", "cylinder", or "human". If you select "human", you need to specify what human phantom to use. The different phantoms can be found in source/phantom/phantom_data. Our current default phantom is "hudfrid.stl". Several more advanced phantoms (e.g. for neurointerventions) are in development. If you select any of the mathematical phantoms (plane or cylinder), then you can tweek settings such as length and radius in settings.phantom.dimension. You can also change the dimension of the patient support table and pad.
2. Patient 
3. Third item
4. Fourth item


In [ ]:
settings.phantom.model="plane"
main(settings=settings)

In [ ]:
main(settings=settings) # högre

In [ ]:
pio.renderers.default = "browser"
main(settings=settings) # högremain(settings=settings) # högre


In [ ]:
pio.renderers
